In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

/home/octav/gitrepos/tum-thesis


In [2]:
model_type = "transformer"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
args.n_epochs = 2000 
args.init_model = f"{model_type}10-onebatch"
args.output_dir = "mol_opt/output_{}".format(args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}".format(args.init_model)
args.init_model = "{}10-onebatch-nonconstrained".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.batch_size = 6

args.annealing_rate = 0.0002
args.connectivity_lambda = 2e-02
args.valency_lambda = 5e-05
args.euler_lambda = 1e-06
args.connectivity = False 
args.valency = False 
args.euler_characteristic_penalty = False
args.scale_lambdas = True
args.conn_penalty_function = "logdet" 
args

Namespace(N_transformer=6, agg_func='sum', annealing_rate=0.0002, batch_norm=False, batch_size=6, conn_penalty_function='logdet', connectivity=False, connectivity_hard=False, connectivity_lambda=0.02, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=False, euler_lambda=1e-06, ffn_activation='LeakyReLU', init_decoder_model='transformer10-onebatch-nonconstrained_decode', init_model='transformer10-onebatch-nonconstrained', linear_out=False, model_type='transformer', n_epochs=2000, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=250, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_transformer10-onebatch', pc_hidden=100, pred_hidden=150, scale_lambdas=True, sinkhorn_entropy=0.1, sinkhorn_max_it=10000, task='qed', tb_logs_dir='mol_opt/logs_transformer10-onebatch', valency=False, valency_hard=False, valency_lambda=5e-05)

In [3]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, False)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)
val_data_loader = None

In [4]:
molopt = main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

No model transformer10-onebatch-nonconstrained found in mol_opt/output_transformer10-onebatch! Starting from scratch.
No model transformer10-onebatch-nonconstrained_decode found in mol_opt/output_transformer10-onebatch! Starting from scratch.
Epoch: 1


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


Epoch 1, train
 train_fgw:4.0421368
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:4.0421368
 train_similarity:0.0000000
 train_penlog:-100.0000000
Epoch duration: 1.4358408451080322
Epoch: 2
Epoch 2, train
 train_fgw:2.1342403
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:2.1342403
 train_similarity:0.0035461
 train_penlog:4.7770635
Epoch duration: 0.7082901000976562
Epoch: 3


/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:394: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


Epoch 3, train
 train_fgw:1.5264831
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:1.5264831
 train_similarity:0.0035461
 train_penlog:4.7770635
Epoch duration: 1.2696022987365723
Epoch: 4
Epoch 4, train
 train_fgw:1.5436201
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:1.5436201
 train_similarity:0.0035461
 train_penlog:4.7770635
Epoch duration: 0.4172646999359131
Epoch: 5
Epoch 5, train
 train_fgw:1.3842363
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:1.3842363
 train_similarity:0.0035461
 train_penlog:4.7770635
Epoch duration: 0.5988864898681641
Epoch: 6
Epoch 6, train
 train_fgw:1.0421227
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:1.0421227
 train_similarity:0.0035461
 train_penlog:4.7770635
Epoch duration: 0.49256181716918945
Epoch: 7
Epoch 7, train
 tra

...

Epoch 1852, train
 train_fgw:0.2494058
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2494058
 train_similarity:0.0272880
 train_penlog:-33.3282332
Epoch duration: 0.21506881713867188
Epoch: 1853
Epoch 1853, train
 train_fgw:0.2520623
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2520623
 train_similarity:0.0293924
 train_penlog:-49.4982982
Epoch duration: 0.19183015823364258
Epoch: 1854
Epoch 1854, train
 train_fgw:0.2604032
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2604032
 train_similarity:0.0541178
 train_penlog:-33.2805610
Epoch duration: 0.2850170135498047
Epoch: 1855
Epoch 1855, train
 train_fgw:0.2539329
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2539329
 train_similarity:0.0600771
 train_penlog:-32.7506387
Epoch duration: 0.404483079910278

Epoch 1885, train
 train_fgw:0.2874344
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2874344
 train_similarity:0.0458798
 train_penlog:-32.6253478
Epoch duration: 0.4692399501800537
Epoch: 1886
Epoch 1886, train
 train_fgw:0.2589732
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2589732
 train_similarity:0.0311407
 train_penlog:-50.2276556
Epoch duration: 0.28232598304748535
Epoch: 1887
Epoch 1887, train
 train_fgw:0.2582404
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2582404
 train_similarity:0.0370823
 train_penlog:-33.1969637
Epoch duration: 0.24028372764587402
Epoch: 1888
Epoch 1888, train
 train_fgw:0.3011286
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.3011286
 train_similarity:0.0520961
 train_penlog:0.3829934
Epoch duration: 0.304689884185791
E

Epoch 1918, train
 train_fgw:0.2382159
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2382159
 train_similarity:0.0240697
 train_penlog:-50.1512494
Epoch duration: 0.21316051483154297
Epoch: 1919
Epoch 1919, train
 train_fgw:0.2464676
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2464676
 train_similarity:0.0356964
 train_penlog:-33.6308307
Epoch duration: 0.21299362182617188
Epoch: 1920
Epoch 1920, train
 train_fgw:0.2701028
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2701028
 train_similarity:0.0276247
 train_penlog:-50.0962968
Epoch duration: 0.2100811004638672
Epoch: 1921
Epoch 1921, train
 train_fgw:0.2447796
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2447796
 train_similarity:0.0604598
 train_penlog:0.5316684
Epoch duration: 0.1955559253692627


Epoch 1951, train
 train_fgw:0.2528170
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2528170
 train_similarity:0.0455799
 train_penlog:-15.9852787
Epoch duration: 0.3150150775909424
Epoch: 1952
Epoch 1952, train
 train_fgw:0.2544498
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2544498
 train_similarity:0.0395602
 train_penlog:-33.1851506
Epoch duration: 0.2572007179260254
Epoch: 1953
Epoch 1953, train
 train_fgw:0.2371642
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2371642
 train_similarity:0.0554530
 train_penlog:-16.2369436
Epoch duration: 0.22876834869384766
Epoch: 1954
Epoch 1954, train
 train_fgw:0.2759369
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2759369
 train_similarity:0.0208411
 train_penlog:-49.1127569
Epoch duration: 0.2128396034240722

Epoch 1984, train
 train_fgw:0.2854292
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2854292
 train_similarity:0.0262616
 train_penlog:-15.8650813
Epoch duration: 0.2080698013305664
Epoch: 1985
Epoch 1985, train
 train_fgw:0.3004584
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.3004584
 train_similarity:0.0419021
 train_penlog:-32.7852290
Epoch duration: 0.27316784858703613
Epoch: 1986
Epoch 1986, train
 train_fgw:0.2835046
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2835046
 train_similarity:0.0221798
 train_penlog:-49.5896218
Epoch duration: 0.3439292907714844
Epoch: 1987
Epoch 1987, train
 train_fgw:0.2922778
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.2922778
 train_similarity:0.0425414
 train_penlog:-50.0841115
Epoch duration: 0.3255817890167236